In [7]:
import pandas as pd

# Assuming 'data.csv' is a CSV file with your data
df = pd.read_csv('../Data/T10YIE.csv')  # You might need to adjust the delimiter

df['Date'] = pd.to_datetime(df['DATE'], format='%Y-%m-%d')

# Replace '.' with NaN to use forward fill
df['Value'] = pd.to_numeric(df['T10YIE'], errors='coerce')

# Now use forward fill to replace NaN with the previous non-NaN value
df['Value'].fillna(method='ffill', inplace=True)

print(df)
df.columns = ["Value"]
df.sort_index(inplace=True)
df.loc['2003-01-01'] = 1.64
df.to_csv('../Data/inflation.csv')
df

            DATE T10YIE       Date  Value
0     2003-01-02   1.64 2003-01-02   1.64
1     2003-01-03   1.62 2003-01-03   1.62
2     2003-01-06   1.63 2003-01-06   1.63
3     2003-01-07   1.62 2003-01-07   1.62
4     2003-01-08   1.71 2003-01-08   1.71
...          ...    ...        ...    ...
5445  2023-11-16   2.27 2023-11-16   2.27
5446  2023-11-17   2.28 2023-11-17   2.28
5447  2023-11-20    2.3 2023-11-20   2.30
5448  2023-11-21   2.25 2023-11-21   2.25
5449  2023-11-22   2.26 2023-11-22   2.26

[5450 rows x 4 columns]


In [7]:
unem = pd.read_csv("..\Data\ICSA.csv")
unem['DATE'] = pd.to_datetime(unem['DATE'], format='%Y-%m-%d')
unem.index = unem['DATE']
unem = unem.drop(['DATE'], axis=1)

In [8]:
unem

,ICSA
DATE,
2000-01-01,286000
2000-01-08,298000
2000-01-15,289000
2000-01-22,284000
2000-01-29,285000
...,...
2022-12-03,216000
2022-12-10,206000
2022-12-17,212000


In [9]:
import numpy as np
def interpolate_weekly_to_daily(weekly_data):
    """
    Interpolate weekly data to daily data using linear interpolation.
    
    :param weekly_data: A pandas Series with dates as index and values as data points.
    :param start_date: The start date for the interpolation range.
    :return: A pandas Series with daily interpolated data.
    """
    # Ensure the index is a DatetimeIndex
    if not isinstance(weekly_data.index, pd.DatetimeIndex):
        weekly_data.index = pd.to_datetime(weekly_data.index)
    
    # Reindex to daily frequency, inserting NaNs for missing days
    daily_data = weekly_data.resample('D').asfreq()
    
    # Perform linear interpolation to fill in the NaNs
    daily_data_interpolated = daily_data.interpolate(method='linear')
    
    return daily_data_interpolated

# Example usage:
# Create a weekly time series (Sundays)
# weekly_dates = pd.date_range(start='2023-01-01', periods=10, freq='W-SUN')
# weekly_values = np.random.rand(len(weekly_dates))  # Random data for example purposes
# weekly_series = pd.Series(weekly_values, index=weekly_dates)

# Interpolate to daily
daily_series = interpolate_weekly_to_daily(unem)

print(daily_series)

# Example usage
# start_date = '2003-01-01'
# end_date = '2022-12-30'

# daily_df = interpolate_weekly_to_daily(unem, start_date, end_date)
# print(daily_df.head())  # Print the first few rows of the daily data


                     ICSA
DATE                     
2000-01-01  286000.000000
2000-01-02  287714.285714
2000-01-03  289428.571429
2000-01-04  291142.857143
2000-01-05  292857.142857
...                   ...
2022-12-27  210000.000000
2022-12-28  209000.000000
2022-12-29  208000.000000
2022-12-30  207000.000000
2022-12-31  206000.000000

[8401 rows x 1 columns]


In [11]:
daily_series.to_csv("../Data/unemployment_claims.csv")

In [62]:
def fetch_financial_data(tickers, start_date='2002-12-31', end_date='2022-12-31'):
    data = yf.download(tickers, start=start_date, end=end_date)
    return data['Close']

# Data cleaning: fill NaN values using forward fill method
def fill_leading_nans_with_one(series):
    first_valid = series.first_valid_index()
    if first_valid is not None:
        series[:first_valid] = 1
    return series

def collect_data(tickers):

    data = fetch_financial_data(tickers)
    data_filled = data.fillna(method='ffill')
    if isinstance(data_filled, pd.DataFrame):
            
        cols = ['^GSPC'] + [col for col in data_filled if col != '^GSPC']
        data_filled = data_filled[cols]
        # data_filled.columns = range(len(data_filled.columns))
    # data_filled = data_filled.pct_change(1)
    return data_filled

In [63]:
import Data_gen

tickers = ["^GSPC", "^IXIC", "^DJI", "JPYUSD=X", "^VIX", "GBPUSD=X", "EURUSD=X"]  
data = collect_data(tickers)
# data = pd.read_csv("Data/fin_data.csv")

# data.to_csv("Data/fin_data.csv")
# data.index = data["Date"]
# data = data.drop(["Date"], axis=1)
# data.columns = range(data.shape[1])
data = data.drop('2002-12-31')
data

[*********************100%***********************]  7 of 7 completed


,^GSPC,EURUSD=X,GBPUSD=X,JPYUSD=X,^DJI,^IXIC,^VIX
Date,,,,,,,
2002-12-31,879.820007,NaN,NaN,0.008425,8341.629883,1335.510010,28.620001
2003-01-01,879.820007,NaN,NaN,0.008424,8341.629883,1335.510010,28.620001
2003-01-02,909.030029,NaN,NaN,0.008319,8607.519531,1384.849976,25.389999
2003-01-03,908.590027,NaN,NaN,0.008356,8601.690430,1387.079956,24.680000
2003-01-06,929.010010,NaN,NaN,0.008378,8773.570312,1421.319946,24.910000
...,...,...,...,...,...,...,...
2022-12-26,3844.820068,1.066780,1.206001,0.007531,33203.929688,10497.860352,20.870001
2022-12-27,3829.250000,1.064022,1.208415,0.007532,33241.558594,10353.230469,21.650000
2022-12-28,3783.219971,1.063717,1.202733,0.007491,32875.710938,10213.290039,22.139999


In [23]:
all.index[0]

'2003-01-01'

In [19]:
daily_series.index = daily_series.index.

In [26]:
all = pd.read_csv("../Data/fin_data.csv", index_col="Date")
all.index = pd.to_datetime(all.index, format='%Y-%m-%d')
all = pd.merge(left=all, right=daily_series, how="left", left_index=True, right_index=True)
all["7"] = all['ICSA']
all = all.drop(["ICSA"], axis=1)
all.to_csv("../Data/fin_data.csv")


In [48]:
er = pd.read_excel("../Data/exchange_Rates.xlsx")
er.index = er['Date']
er.index = pd.to_datetime(er.index)
er.index = er.index.normalize()
er.drop('Date',axis=1, inplace=True)
er

,EURUSD,GBPUSD
Date,,
2003-01-01 23:58:00,1.047300,1.609300
2003-02-01 23:58:00,1.035300,1.592300
2003-03-01 23:58:00,1.042300,1.610700
2003-04-01 23:58:00,1.042600,1.610200
2003-05-01 23:58:00,1.041900,1.609800
...,...,...
12/26/2022 23:58:00,1.063970,1.208290
12/27/2022 23:58:00,1.063685,1.202615
12/28/2022 23:58:00,1.062925,1.202970


In [7]:
import pandas as pd
import ARXT
import numpy as np
def get_data():

    # tickers = ["^GSPC", "^IXIC", "^DJI","JPYUSD=X", "^VIX", "GBPUSD=X", "EURUSD=X] # , "GBPUSD=X", "EURUSD=X",
    # data = Data_gen.collect_data(tickers)
    data = pd.read_csv("..\\Data\\fin_data.csv")
    # data.to_csv("Data/fin_data.csv")
    data.index = data["Date"]
    data = data.drop(["Date"], axis=1)
    data.columns = range(data.shape[1])

    return(data)

data = get_data()

# Now use forward fill to replace NaN with the previous non-NaN value
data = data.fillna(method='ffill', inplace=True)
data.to_csv("..\\Data\\fin_data.csv")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (9,) + inhomogeneous part.

In [17]:
print(len(ts_valid))
ts_valid[0]

9


array([-1.66204048, -1.66721319, -1.64362568, ...,  2.64251227,
        2.65729144,  2.72832024])

In [97]:
# merged_df = pd.merge(data, er, how='left', left_on=data.index, right_on=er.index, left_index=True)
merged_df = pd.merge(data, er, how='left',left_index=True, right_index=True)
# merged_df.index = pd.to_datetime(merged_df.index).normalize
# merged_df.index = merged_df['key_0']
merged_df = pd.merge(merged_df, daily_series, how='left',left_index=True, right_index=True)

merged_df['Inflation'] = df

merged_df = merged_df.drop(["EURUSD=X","GBPUSD=X"], axis=1)
merged_df

,^GSPC,JPYUSD=X,^DJI,^IXIC,^VIX,EURUSD,GBPUSD,ICSA,Inflation
Date,,,,,,,,,
2003-01-01,879.820007,0.008424,8341.629883,1335.510010,28.620001,1.047300,1.609300,399857.142857,1.64
2003-01-02,909.030029,0.008319,8607.519531,1384.849976,25.389999,1.076700,1.646900,397571.428571,1.64
2003-01-03,908.590027,0.008356,8601.690430,1387.079956,24.680000,1.080000,1.575700,395285.714286,1.62
2003-01-06,929.010010,0.008378,8773.570312,1421.319946,24.910000,1.172200,1.626100,388714.285714,1.63
2003-01-07,922.929993,0.008318,8740.589844,1431.569946,25.129999,1.156400,1.663300,386571.428571,1.62
...,...,...,...,...,...,...,...,...,...
2022-12-26,3844.820068,0.007531,33203.929688,10497.860352,20.870001,1.063970,1.208290,211000.000000,2.20
2022-12-27,3829.250000,0.007532,33241.558594,10353.230469,21.650000,1.063685,1.202615,210000.000000,2.26
2022-12-28,3783.219971,0.007491,32875.710938,10213.290039,22.139999,1.062925,1.202970,209000.000000,2.28


In [2]:
merged_df['ICSA'] = merged_df['ICSA']/1000
merged_df.to_csv("..\\Data\\fin_data.csv")

NameError: name 'pd' is not defined

In [5]:
import pandas as pd
data = pd.read_csv("..\\Data\\fin_data.csv")
data.index = data["Date"]
data = data.drop(["Date"], axis=1)
print(data.pct_change())
print(data.diff())

                   0         1         2         3         4         5   
Date                                                                     
2003-01-01       NaN       NaN       NaN       NaN       NaN       NaN  \
2003-01-02  0.033200 -0.012478  0.031875  0.036945 -0.112858  0.028072   
2003-01-03 -0.000484  0.004512 -0.000677  0.001610 -0.027964  0.003065   
2003-01-06  0.022474  0.002597  0.019982  0.024685  0.009319  0.085370   
2003-01-07 -0.006545 -0.007154 -0.003759  0.007212  0.008832 -0.013479   
...              ...       ...       ...       ...       ...       ...   
2022-12-26  0.000000 -0.003261  0.000000  0.000000  0.000000 -0.002700   
2022-12-27 -0.004050  0.000196  0.001133 -0.013777  0.037374 -0.000268   
2022-12-28 -0.012021 -0.005409 -0.011006 -0.013517  0.022633 -0.000714   
2022-12-29  0.017461 -0.004089  0.010497  0.025927 -0.031617  0.002940   
2022-12-30 -0.002541  0.008411 -0.002214 -0.001108  0.010728  0.006097   

                   6         7       

In [6]:
data

,0,1,2,3,4,5,6,7,8
Date,,,,,,,,,
2003-01-01,879.820007,0.008424,8341.629883,1335.510010,28.620001,1.047300,1.609300,3.998571e-07,1.64
2003-01-02,909.030029,0.008319,8607.519531,1384.849976,25.389999,1.076700,1.646900,3.975714e-07,1.64
2003-01-03,908.590027,0.008356,8601.690430,1387.079956,24.680000,1.080000,1.575700,3.952857e-07,1.62
2003-01-06,929.010010,0.008378,8773.570312,1421.319946,24.910000,1.172200,1.626100,3.887143e-07,1.63
2003-01-07,922.929993,0.008318,8740.589844,1431.569946,25.129999,1.156400,1.663300,3.865714e-07,1.62
...,...,...,...,...,...,...,...,...,...
2022-12-26,3844.820068,0.007531,33203.929688,10497.860352,20.870001,1.063970,1.208290,2.110000e-07,2.20
2022-12-27,3829.250000,0.007532,33241.558594,10353.230469,21.650000,1.063685,1.202615,2.100000e-07,2.26
2022-12-28,3783.219971,0.007491,32875.710938,10213.290039,22.139999,1.062925,1.202970,2.090000e-07,2.28
